# 1 - To start with

## Drive folder structure
Below is the structure of our drive, to clarify the paths variables we used throughout the notebook.

```
.
├── drive 
│   ├── MyDrive             
|   |         ├── Cleaned_data
|   |         |            ├── cleaned-quotes-2019.csv.bz2
|   |         |            ├── ...
|   |         ├── Processed_datasets
|   |         |            ├── processed-cleaned-quotes-2019.csv.bz2
|   |         |            ├── ...
|   |         ├── Project_datasets
|   |         |            ├── speakers_attributes.parquet
|   |         ├── Quotebank
|   |         |            ├── quotes-2019.json.bz2
|   |         |            ├── ...
|   |         ├── gender_extraction
|   |         └── ...
│   └── ...                 
└── ...                
```

## Install

In [ ]:
!pip install pandas==1.0.5

In [ ]:
!pip install pyarrow

## Imports

In [ ]:
import os

import pandas as pd
import numpy as np
import requests
import re
import itertools
import collections
import nltk
import networkx

from glob import glob
from urllib.parse import urlparse
from dateutil.parser import parse
from nltk.corpus import stopwords


## Mounting Google Drive and getting the python files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Load python file for gender extraction

In [ ]:
 ! cp drive/MyDrive/ADA/gender_extraction.py .

## Loading the data

## Cleaned data
Here we can load our processed and cleaned data. We can see that for year 2019, we have roughly 57'000'000 quotes, so this is enough for machine learning tasks. 

In [ ]:
path_to_read_from = '/content/drive/MyDrive/ADA/Processed_datasets'
file_name = 'processed-cleaned-quotes-2019.csv.bz2' 

chunksize = 10000 # = 10k lines

# Load dataframe in reader
df_reader_cleaned = pd.read_csv(os.path.join(path_to_read_from, file_name), compression ='bz2', chunksize = chunksize)

In [ ]:
valid_quotes_number = 0
for chunk in df_reader_cleaned:
  valid_quotes_number += chunk.shape[0]

print("There are {} quotes.".format(valid_quotes_number))


## Some statistics on speakers attributes
For our analysis, we've only selected only binary genders, males and females, as they make up to nearly 100 % of the total speaker pool.

In [ ]:
path_parquet = '/content/drive/MyDrive/ADA/Project_datasets/speaker_attributes.parquet'

# Find all the files absolute path
path_parquet_files = path_parquet + '/part*'
files = sorted(glob(path_parquet_files))

# Find all the files relative path USELESS
os.chdir(path_parquet) # might be dangerous because set the current working directory for the entire notebook cells
names = glob("part*")

nb_males=0
nb_females=0

for f in files: 
  df_speakers = pd.read_parquet(f, engine = 'pyarrow',columns = ['gender'])
  nb_males = nb_males + df_speakers['gender'].value_counts().values[0]
  nb_females = nb_females + df_speakers['gender'].value_counts().values[1]

proportion_males = nb_males/(nb_males+nb_females)
proportion_females = nb_females/(nb_males+nb_females)

print("The proportion of female speakers overall is {}% and of male is {}%.".format(proportion_females*100, proportion_males*100))

# 2 - Cleaning the data

## General cleaning functions

In [ ]:
def clean_chunk(chunk):

  """Filter out the rows in chunk which:
  - have first speaker attribution probability less than 0.5 (empirically)
  - have a None speaker or an empty QID
  Delete the unnecessary columns for our analysis
  """


  # Select the highest probability
  chunk['h_probas'] = chunk.apply(lambda p: p['probas'][0][1], axis=1)
  # Select the corresponding speaker
  chunk['h_probas_speaker'] = chunk.apply(lambda p : p['probas'][0][0], axis=1)
  # Select the associated speaker QID
  chunk['qids'] = chunk.apply(lambda p : p['qids'][0] if len(p['qids']) >= 1 else p['qids'], axis=1)
  

# Filter the rows
  chunk = chunk.loc[(chunk['speaker'] != 'None') &
                    (chunk['h_probas'] > '0.5') &
                    (chunk['qids'].astype(str) != '[]')
                    ]

  return chunk.drop(labels=['probas','h_probas_speaker','quoteID', 'phase'], axis=1)

## Quotation specific cleaning

In [ ]:
def is_date(string, fuzzy=False):
  """
  Determine whether a given string can be parsed as a date
  If it can be, return True, otherwise return False.
  @Param : - string : string to be parsed
           - fuzzy : boolean allowing fuzzy parsing
  @Return : - boolean : whether string can be parsed as a date or not"""
  try:
    parse(string, fuzzy = fuzzy)
    return True
  except ValueError:
    return False

In [ ]:
def tagfree(string):
  """
  Returns whether a given string is url tag free or not
  """

  # If there is an url in the quotation then it might not be a "real" quotation but rather headlines
  if (re.search('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string) != None):
    return False
    
  else:
    return True

# Uncomment below. The result should be False ! (extracted from real quotes)
# tagfree("Thomas Tuchel makes me think of Louis van Gaal More Stories < a href = `http://ads.xyz.ng/www/delivery/ck.php?n=a322e25b&cb=787667145' target =' _ blank' > < img src = `http://ads.xyz.ng/www/delivery/avw.php?zoneid=7&cb=565603679&n=a322e25b' border =' 0' alt =")

In [ ]:
def html_tagfree(string):
  """Takes a string and return wether this string is html tag free or not"""

  if(re.search('<.+?>', string) != None): 
    return False
  else :
    return True


In [ ]:
def clean_quotations(chunk):
  """
  Filters out the incorrect quotations
  """
  # Find all the quotations that are not a date
  date_mask = chunk.apply(lambda p: not is_date(p['quotation']), axis = 1)
  # Find all the quotations that are tag free
  mask = chunk.apply(lambda p: tagfree(p['quotation']), axis = 1)

  chunk = chunk.loc[(date_mask & mask) , :]            

  return chunk

## Topic selection functions

In [ ]:
def talks_about(url_list, key_words_list):
  """Check whether some keywords are inside the urls to retrieve the topics of interest
  @Param : - url_list : list of urls (strings)
           - key_words_list : list of key words (string)
  @Return : True if at least on key word was found in in at least on url, False otherwise """

  for url in url_list :
    if any(key in url for key in key_words_list):
      return True
  
  return False

In [ ]:
def topic_selection(chunk, keywords):
  """Select the rows which topic is in keywords list:
  @Param : - chunk : pandas DatFrame to filter
           - Keywords : a list of keywords"""

  topic_relevant_index = chunk.urls.apply(lambda p : talks_about(p, keywords))
  
  return chunk.drop(chunk[~topic_relevant_index].index)

## Write to file
We use several steps to clean our data 
- First we make general checks on the rows, eg. removing None speakers and thresholding the speaker probability to 0.5, empirically.
- Then we remove the quotes that can be parsed as dates and that contain urls
- Finally we select the quotes that are topic relevant for us, using a lexical field 🌳 : ecology, biodiversity, environment, global-warming,ecosystem, sustainability. Climate has been omitted since it can also referred to a mood (climate of anger, etc...).

In [ ]:
# QUOTES
path_quotes = '/content/drive/MyDrive/ADA/Quotebank'
file_quotes = 'quotes-2019.json.bz2' # Leave it zipped !
 
chunksize = 100000 # = 10k
# chunk = morceau

# Load dataframe in reader
df_quotes_reader = pd.read_json(os.path.join(path_quotes, file_quotes), lines=True, compression ='bz2', chunksize = chunksize)


In [ ]:
# Here you need to specify the folder path to write to (ie. you already need to create the folders)
path_out = '/content/drive/MyDrive/ADA/Cleaned_data'

# Name of the file (depends on the file we read from with df_quotes_reader)
cleaned_file_name = 'non_header-cleaned-' + file_quotes[:-8] + 'csv.bz2'
header = True

for chunk in df_quotes_reader :

  # Process chunk
  chunk = clean_chunk(chunk) # your cleaning function
  chunk = clean_quotations(chunk)
  chunk = topic_selection(chunk, ['ecology','biodiversity','environment','global-warming','ecosystem','sustainability'])

  # Write to file in memory, avoiding writing the header each time
  if header:
    chunk.to_csv(path_or_buf = os.path.join(path_out, cleaned_file_name), compression = 'bz2', mode = 'a') # mode a : appends at the end of the same dataframe
    header = False
  else :
    chunk.to_csv(path_or_buf = os.path.join(path_out, cleaned_file_name), compression = 'bz2', mode = 'a', header = False)


#  3 - Enrich data set 
Enrich data set with speaker gender. Since we need to merge two bug data frames, we enrich our dataset after the first cleaning round. 

## Gender matching functions

In [ ]:
def match_and_merge(chunk, gender):
  """Match a speaker and his/her gender, considering only binary genders
  @Param : - chunk : pandas DataFrame
           - gender : binary gender DataFrame
  @Return :  chunk containing speakers and their gender """

  # Merge both data frames in an inner joint fashion
  chunk = chunk.merge(gender, left_on='qids', right_on='id', how='inner')
  
  return chunk

## Extract gender from speaker attributes
We aim at filtering out all the speakers whose gender is not binary male or female.

In [ ]:
from gender_extraction import extract_gender

extract_gender()

## Write to file

In [ ]:
# GENDER

path_gender = '/content/drive/MyDrive/ADA/Processed_datasets/Gender/speakers-genders.csv'
gender = pd.read_csv(path_gender, compression='bz2', index_col = 0)

In [ ]:
# CLEANED QUOTES
path_quotes = '/content/drive/MyDrive/ADA/Cleaned_data'
file_quotes = 'cleaned-quotes-2019.csv.bz2' # weird name

 
chunksize = 1000 # = 10k
# chunk = morceau

# Load dataframe in reader
df_quotes_reader = pd.read_csv(os.path.join(path_quotes, file_quotes), index_col = 0, compression ='bz2', chunksize = chunksize)

In [ ]:
 path_out = '/content/drive/MyDrive/ADA/Processed_datasets'

# Name of the file (depends on the file we read from with df_reader)
cleaned_file_name = 'processed-' + file_quotes
header = True

for chunk in df_quotes_reader :

  # Process chunk
  enriched_chunk = match_and_merge(chunk, gender) 
  
  # Write to file in memory
  if header :
    enriched_chunk.to_csv(path_or_buf = os.path.join(path_out, cleaned_file_name), compression = 'bz2', mode = 'a') # mode a : appends at the end of the same dataframe
    header = False
  else :
    chunk.to_csv(path_or_buf = os.path.join(path_out, cleaned_file_name), compression = 'bz2', mode = 'a', header = False)

## Some statistical analysis on raw data for dataset of 2019
Here we present some filtering we've done, retrieving the quotes we discarded.

In [ ]:
path_to_read_from = '/content/drive/MyDrive/ADA/Quotebank'
file_name = 'quotes-2019.json.bz2' 

chunksize = 1000 # = 10k

# Load dataframe in reader
df_reader_raw = pd.read_json(os.path.join(path_to_read_from, file_name), lines = True, compression ='bz2', chunksize = chunksize)

In [ ]:
## Print quote with urls in it because they cannot be considered as quotes, but rather as kind of headlines

for chunk in df_reader_raw :
  mask = chunk.apply(lambda p: not tagfree(p['quotation']), axis = 1)
  if(chunk.loc[mask,'quotation'].values.size != 0):
    print(chunk.loc[mask,'quotation'].values)
    break



In [ ]:
## Print quote with date in it

for chunk in df_reader_raw :

  date_mask = chunk.apply(lambda p: is_date(p['quotation']), axis = 1)
  if(chunk.loc[date_mask,'quotation'].values.size != 0):
    print(chunk.loc[date_mask,'quotation'].values)
    break


In [ ]:
## Print quote with html tags in it. We will clean these quotes in a second step.

for chunk in df_reader_raw :

  mask = chunk.apply(lambda p: not html_tagfree(p['quotation']), axis = 1)
  if(chunk.loc[mask,'quotation'].values.size != 0):
    print("Raw quotation:\n",chunk.loc[mask,'quotation'].values[0])
    print("Cleaned quotation :\n", html_cleaning(chunk.loc[mask, 'quotation'].values[0]))
    break



# 4 - Deeper quotes processing for word frequency and sentiment analysis
We need to remove the html tags and hashtags words that are left, and then to lower case all words, and split the quotes into a set of unique words.
Word frequency and sentiment analysis seem reasonable here, since we have enough data and 


In [ ]:
def html_cleaning(string):
  """Remove the html and # tags from the string, using a regex"""
  
  # from https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/calculate-tweet-word-frequencies-in-python/
  CLEANR = re.compile('([^0-9A-Za-z \t])|(\w+:\/\/\S+)')
  cleantext = re.sub(CLEANR, '', string)
  return cleantext

In [ ]:
def process_quote(string):
  """Return all the unique words presents in a string in lower case. """
  return list(set(string.lower().split()))

# 5 - Methods for word frequency analysis

We'll be using nltk library which provides us with tools for sentiment analysis and initial word frequency analysis

In [ ]:
path_to_read_from = '/content/drive/MyDrive/ADA/Cleaned_data'
file_name = 'cleaned-quotes-2019.csv.bz2' 

chunksize = 10000 # = 10k lines

# Load dataframe in reader
df_reader_cleaned = pd.read_csv(os.path.join(path_to_read_from, file_name), index_col=0, compression ='bz2', chunksize = chunksize)

In [ ]:
# Let's provide an example for one chunk
for chunk in df_reader_cleaned :
  chunk['quotation'] = chunk.apply(lambda p : html_cleaning(p['quotation']), axis = 1)
  chunk['quotation'] = chunk.apply(lambda p : process_quote(p['quotation']), axis = 1)

  # Retrieve all words
  all_words = list(itertools.chain(*chunk.quotation.values))
  initial_count = collections.Counter(all_words)
  print(initial_count.most_common(15))
  print("We filter out all the stopwords such as the, a,...")

  # Download stop words
  nltk.download('stopwords')
  stop_words = set(stopwords.words('english'))

  # Filter out stop words
  quotes_nsw = [[word for word in quote if not word in stop_words]
              for quote in chunk.quotation.values]

  all_words_nsw = list(itertools.chain(*quotes_nsw))
  counts = collections.Counter(all_words_nsw)
  print(counts.most_common(15))
  
  break
